In [21]:
import cv2
import numpy as np
import imutils
from imutils import perspective
from imutils import contours

def order_pts_old(pts):
    rect = np.zeros((4,2),dtype = "float32")
    s = pts.sum(axis=1)
    rect[0] = pts[np.argmin(s)]
    rect[2] = pts[np.argmax(s)]
    
    diff = pts.diff(axis=1)
    rect[1] = pts[np.argmin(diff)]
    rect[3] = pts[np.argmax(diff)]
    
    return rect

# Better implemention of order_pts_old() funtion

In [22]:
from scipy.spatial import distance as dist
import numpy as np
import cv2

def order_pts(pts):
    xSorted = pts[np.argsort(pts[:,0]),:]
    leftMost = xSorted[:2,:]
    rightMost = xSorted[2:,:]
    leftMost = leftMost[np.argsort(leftMost[:,1]),:]
    (tl,bl) = leftMost
    D = dist.cdist(tl[np.newaxis], rightmost, "euclidean")[0]
    (br,tr) = rightMost[np.argsort(D)[::-1],:]
    return np.array([tl,tr,br,bl], dtype="float32")


In [23]:
cap = cv2.VideoCapture(0)
while(True):
    ret,frame = cap.read()
    gray = cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
    blur = cv2.GaussianBlur(gray,(7,7),0)
    
    edged = cv2.Canny(blur,50,100)
    edged = cv2.dilate(edged, None, iterations=1)
    edged = cv2.erode(edged, None, iterations=1)
    
    cnts = cv2.findContours(edged.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    cnts = imutils.grab_contours(cnts)
    
    (cnts,_) = contours.sort_contours(cnts)
    colors = ((0,0,255),(240,0,159),(255,0,0),(255,255,0))
    
    for (i,c) in enumerate(cnts):
        if(cv2.contourArea(c) < 100):
            continue
            
        box = cv2.minAreaRect(c)
        box = cv2.boxPoints(box)
        box = np.array(box, dtype="int")
        
        cv2.drawContours(frame,[c],-1,(0,255,0),2)
        cv2.drawContours(frame,[box],-1,(0,255,0),2)
        
        rect = perspective.order_points(box)
        
        for ((x,y),color) in zip(rect, colors):
            cv2.circle(frame, (int(x),int(y)), 5, color, -1)
        
        cv2.putText(frame, "Object #{}".format(i+1),(int(rect[0][0] - 15), int(rect[0][1] - 15)),cv2.FONT_HERSHEY_SIMPLEX, 0.555,(255,255,255), 2)
        
    
    cv2.imshow("image",frame)
    if(cv2.waitKey(1) == 13):
        break
cap.release()
cv2.destroyAllWindows()